### Este notebook Genera una carpeta con todas las imágenes que tienen masas o calcificaciones.

In [1]:
import os
import shutil
import pandas as pd

# Rutas a las carpetas y archivos CSV
csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_with_image_names.csv'
source_images_dir = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/original/Vindrmammo/images'
destination_images_dir = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/subset_datos/Images_Mass_Calc'

# Cargar el archivo CSV
filtered_df = pd.read_csv(csv_path)

# Eliminar duplicados basados en 'image_id', manteniendo solo la primera ocurrencia
filtered_unique_df = filtered_df.drop_duplicates(subset='image_id', keep='first')

# Función para copiar la imagen a la carpeta de destino manteniendo la estructura de directorios
def copy_image(image_id, study_id, source_dir, destination_dir):
    # Ruta de la imagen en el directorio original
    source_image_path = os.path.join(source_dir, study_id, image_id + '.dicom')
    
    # Crear la misma estructura de carpetas en el directorio de destino
    destination_study_dir = os.path.join(destination_dir, study_id)
    
    # Si no existe la carpeta de destino, crearla
    os.makedirs(destination_study_dir, exist_ok=True)
    
    # Ruta de la imagen en la carpeta de destino
    destination_image_path = os.path.join(destination_study_dir, image_id + '.dicom')
    
    # Copiar la imagen si no existe ya en la carpeta de destino
    if not os.path.exists(destination_image_path):
        shutil.copy2(source_image_path, destination_image_path)
        print(f"Imagen {image_id} copiada a {destination_image_path}")
    else:
        print(f"Imagen {image_id} ya existe en la carpeta de destino, omitiendo...")

# Recorrer las imágenes únicas y copiarlas a la carpeta de destino
for index, row in filtered_unique_df.iterrows():
    image_id = row['image_id']
    study_id = row['study_id']
    
    # Copiar la imagen a la carpeta de destino manteniendo la estructura
    copy_image(image_id, study_id, source_images_dir, destination_images_dir)

print("Proceso de copia completado.")


Imagen 001ade2a3cb53fd808bd2856a0df5413 copiada a /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/subset_datos/Images_Mass_Calc/8b202f831f93639ebcb9cccccec03b09/001ade2a3cb53fd808bd2856a0df5413.dicom
Imagen 002074412a8fc178c271fb93b55c3e29 copiada a /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/subset_datos/Images_Mass_Calc/b9c15e629cf02f501acbc194420d749c/002074412a8fc178c271fb93b55c3e29.dicom
Imagen 002460132586dc0c7b88a59dce6e77bd copiada a /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/subset_datos/Images_Mass_Calc/76c60021bdad852f982497bb6dfa6cf2/002460132586dc0c7b88a59dce6e77bd.dicom
Imagen 00857417d07096982013956033da1f75 copiada a /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/subset_datos/Images_Mass_Calc/14a63118d94643a3d65b32039a20b351/00857417d07096982013956033da1f75.dicom
Imagen 008c66563c73b2f5b8e42915b2cd6af5 copiada a /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/pr

In [2]:
# Función para contar el número de archivos .dicom en una carpeta (incluyendo subcarpetas)
def count_dicom_files(directory):
    dicom_count = 0
    for root, dirs, files in os.walk(directory):
        dicom_count += len([f for f in files if f.endswith('.dicom')])
    return dicom_count

# Contar cuántas imágenes .dicom se copiaron en total
total_images = count_dicom_files(destination_images_dir)

print(f"Proceso de copia completado. Total de imágenes copiadas: {total_images}")

Proceso de copia completado. Total de imágenes copiadas: 1255


In [ ]:
### Ya tenemos las imagenes copiadas a la carpeta en cuestión.
